# Movie Data Analysis <a id='intro'></a>

## Introduction <a id='intro'></a>
In this project, I will work with data from the entertainment industry with records on movies and shows. The research will focus on the "Golden Age" of television, which began in 1999 with the release of *The Sopranos* and is still ongoing.

The aim of this project is to investigate how the number of votes a title receives impacts its ratings. The assumption is that highly-rated shows (we will focus on TV shows, ignoring movies) released during the "Golden Age" of television also have the most votes.

First, I'll evaluate the quality of the data and see whether its issues are significant. Then, during data preprocessing, I will account for the most critical problems.
 
My project will consist of three stages:
 1. Data overview
 2. Data preprocessing
 3. Data analysis

## Data Overview <a id='data_review'></a>

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/movies_and_shows.csv'); df.head(10)

,name,Character,r0le,TITLE,Type,release Year,genres,imdb sc0re,imdb v0tes
0,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
1,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
2,Albert Brooks,Tom,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
3,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
4,Cybill Shepherd,Betsy,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
5,Peter Boyle,Wizard,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
6,Leonard Harris,Senator Charles Palantine,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
7,Diahnne Abbott,Concession Girl,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
8,Gino Ardito,Policeman at Rally,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
9,Martin Scorsese,Passenger Watching Silhouette,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0


The imported data contains nine columns. The majority store the same data type: object. The only exceptions are `'release Year'` (int64 type), `'imdb sc0re'` (float64 type) and `'imdb v0tes'` (float64 type). Scores and votes will be used in our analysis, so it's important to verify that they are present in the dataframe in the appropriate numeric format. Three columns (`'TITLE'`, `'imdb sc0re'` and `'imdb v0tes'`) have missing values.

According to the documentation:
- `'name'` — actor/director's name and last name
- `'Character'` — character played (for actors)
- `'r0le '` — the person's contribution to the title (it can be in the capacity of either actor or director)
- `'TITLE '` — title of the movie (show)
- `'  Type'` — show or movie
- `'release Year'` — year when movie (show) was released
- `'genres'` — list of genres under which the movie (show) falls
- `'imdb sc0re'` — score on IMDb
- `'imdb v0tes'` — votes on IMDb

We can see three issues with the column names:
1. Some names are uppercase, while others are lowercase.
2. There are names containing whitespace.
3. A few column names have digit '0' instead of letter 'o'. 


Here is the general information about the table:

In [2]:
df.describe()

,release Year,imdb sc0re,imdb v0tes
count,85579.000000,80970.000000,8.085300e+04
mean,2015.879994,6.425877,5.978271e+04
std,7.724668,1.122655,1.846287e+05
min,1954.000000,1.500000,5.000000e+00
25%,2015.000000,5.700000,1.266000e+03
50%,2018.000000,6.500000,5.448000e+03
75%,2021.000000,7.200000,3.360900e+04
max,2022.000000,9.500000,2.294231e+06


Each row in the table stores data about a movie or show. The columns can be divided into two categories: the first is about the roles held by different people who worked on the movie or show (role, name of the actor or director, and character if the row is about an actor); the second category is information about the movie or show itself (title, release year, genre, imdb figures).

It's clear that there is sufficient data to do the analysis and evaluate our assumption. However, to move forward, we need to preprocess the data.

## Data Preprocessing <a id='data_preprocessing'></a>
I will correct the formatting in the column headers and deal with the missing values before checking whether there are duplicates in the data.

### Rename Columns

To start, I will first rename the columns for better readability.

In [3]:
df.columns

Index(['   name', 'Character', 'r0le', 'TITLE', '  Type', 'release Year',
       'genres', 'imdb sc0re', 'imdb v0tes'],
      dtype='object')

In [4]:
df = df.rename(columns = {'   name':'name', 'Character':'character', 'r0le':'role', 
                          'TITLE':'title', 'release Year':'release_year','imdb sc0re':'imdb_score', 
                          'imdb v0tes':'imdb_votes', '  Type':'type'})

In [5]:
df.columns

Index(['name', 'character', 'role', 'title', 'type', 'release_year', 'genres',
       'imdb_score', 'imdb_votes'],
      dtype='object')

### Find Missing Values
Then, I will find the number of missing values in the table.

In [6]:
df.isna().sum()

name               0
character          0
role               0
title              1
type               0
release_year       0
genres             0
imdb_score      4609
imdb_votes      4726
dtype: int64

We can see the missing values in several columns. While the missing value in `'title'` isn't critical, missing values in `'imdb_score'` and `'imdb_votes'` affect around 6% of the data, which could impact our analysis. To ensure data integrity, we'll drop all rows with missing values.

Now the data doesn't contain any more missing values.

In [7]:
df = df.dropna()
df.isna().sum()

name            0
character       0
role            0
title           0
type            0
release_year    0
genres          0
imdb_score      0
imdb_votes      0
dtype: int64

### Duplicates <a id='duplicates'></a>
Now, I will find the number of duplicate rows in the table:

In [8]:
df.duplicated().sum()

6994

Review the duplicate rows to determine if removing them would distort our dataset.

In [9]:
df[df.duplicated()].tail()

,name,character,role,title,type,release_year,genres,imdb_score,imdb_votes
85569,Jessica Cediel,Liliana Navarro,ACTOR,Lokillo,MOVIE,2021,['comedy'],3.8,68.0
85570,Javier Gardeaz?­bal,"Agust??n ""Peluca"" Ort??z",ACTOR,Lokillo,MOVIE,2021,['comedy'],3.8,68.0
85571,Carla Giraldo,Valery Reinoso,ACTOR,Lokillo,MOVIE,2021,['comedy'],3.8,68.0
85572,Ana Mar??a S?­nchez,Lourdes,ACTOR,Lokillo,MOVIE,2021,['comedy'],3.8,68.0
85577,Isabel Gaona,Cacica,ACTOR,Lokillo,MOVIE,2021,['comedy'],3.8,68.0


There are two clear duplicates in the printed rows, meaning I can safely remove them. The code below confirms there are no more duplicated rows.

In [10]:
df = df.drop_duplicates().reset_index()
df.duplicated().sum()

0

Now I will get rid of implicit duplicates in the `'type'` column, since there are rows that have the string `'SHOW'` written in different ways.

In [11]:
# viewing unique type names
df['type'].unique()

array(['MOVIE', 'the movie', 'tv show', 'shows', 'movies', 'tv shows',
       'tv series', 'tv', 'SHOW'], dtype=object)

In [12]:
# function for replacing implicit duplicates
def replace_wrong_show(wrong_shows_list, correct_shows):
    for i in wrong_shows_list:
        df['type'] = df['type'].replace(i, correct_shows)
    return df

In [13]:
# removing implicit duplicates
wrong_shows_list = ['shows', 'tv show', 'tv shows', 'tv series', 'tv']
replace_wrong_show(wrong_shows_list, 'SHOW').head()

,index,name,character,role,title,type,release_year,genres,imdb_score,imdb_votes
0,0,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
1,1,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
2,2,Albert Brooks,Tom,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
3,3,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
4,4,Cybill Shepherd,Betsy,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0


Make sure the duplicate names are removed. Print the list of unique values from the `'type'` column:

In [14]:
# viewing unique genre names
df['type'].unique()

array(['MOVIE', 'the movie', 'SHOW', 'movies'], dtype=object)

We detected three issues with the data:

- Incorrect header styles
- Missing values
- Duplicate rows and implicit duplicates

The headers have been cleaned up to make processing the table simpler.

All rows with missing values have been removed. 

The absence of duplicates will make the results more precise and easier to understand.

Now we can move on to our analysis of the prepared data.

## Data analysis <a id='hypotheses'></a>

Based on the previous project stages, I can now define how the assumption will be checked. To do this, I'll calculate the average amount of votes for each score (this data is available in the `imdb_score` and `imdb_votes` columns), and then check how these averages relate to each other. If the averages for shows with the highest scores are bigger than those for shows with lower scores, the assumption appears to be true.

I will do so by filtering the dataframe and only include shows released in 1999 or later, keeping only titles published in 1999 or later in our dataframe before filtering the table to only contain shows. Additionally, for simplicity, the scores that are to be grouped will be rounded. For instance, titles with scores like 7.8, 8.1, and 8.3 will all be placed in the same bucket with a score of 8.

In [15]:
release = df[df['release_year'] >= 1999]
release = release[release['type'] == 'SHOW']
# rounding column with scores
release['imdb_score'] = release['imdb_score'].round()
#checking the outcome with tail()
release.tail()

,index,name,character,role,title,type,release_year,genres,imdb_score,imdb_votes
73831,85433,Maneerat Kam-Uan,Ae,ACTOR,Let's Eat,SHOW,2021,"['drama', 'comedy']",8.0,5.0
73832,85434,Rudklao Amratisha,unknown,ACTOR,Let's Eat,SHOW,2021,"['drama', 'comedy']",8.0,5.0
73833,85435,Jaturong Mokjok,unknown,ACTOR,Let's Eat,SHOW,2021,"['drama', 'comedy']",8.0,5.0
73834,85436,Pisamai Wilaisak,unknown,ACTOR,Let's Eat,SHOW,2021,"['drama', 'comedy']",8.0,5.0
73835,85437,Sarawut Wichiensarn,unknown,DIRECTOR,Let's Eat,SHOW,2021,"['drama', 'comedy']",8.0,5.0


It is now time to identify outliers based on the number of votes.

In [16]:
score_counts = release.groupby('imdb_score')['imdb_votes'].count()
print(score_counts)

imdb_score
2.0       24
3.0       27
4.0      180
5.0      592
6.0     2494
7.0     4706
8.0     4842
9.0      557
10.0       8
Name: imdb_votes, dtype: int64


Based on the aggregation performed, it is evident that scores 2 (24 voted shows), 3 (27 voted shows), and 10 (only 8 voted shows) are outliers. There isn't enough data for these scores for the average number of votes to be meaningful.

To obtain the mean numbers of votes for the selected scores (we identified a range of 4-9 as acceptable), I will use conditional filtering and grouping.

In [17]:
# filtering dataframe (scores to be in the range 4-9)
filtered = release[(release['imdb_score'] >= 4) & (release['imdb_score'] <= 9)]
# group scores and corresponding average number of votes, reset index and print the result
mean = filtered.groupby('imdb_score')['imdb_votes'].mean().reset_index(); mean

,imdb_score,imdb_votes
0,4.0,5277.583333
1,5.0,3143.942568
2,6.0,3481.717322
3,7.0,8727.068211
4,8.0,30299.460967
5,9.0,126904.109515


Now, I'll round the column with the averages, rename both columns, and print the dataframe in descending order.

In [18]:
# round column with averages
mean['imdb_votes'] = mean['imdb_votes'].round()
# rename columns
mean = mean.rename(columns={'imdb_score': 'Score', 'imdb_votes': 'Votes'})
# print dataframe in descending order
mean_sorted = mean.sort_values(by='Votes', ascending=False)
mean_sorted

if 'imdb_votes' in filtered.columns:
         mean = filtered.groupby('imdb_score')['imdb_votes'].mean().reset_index()
         print(mean.head())

   imdb_score    imdb_votes
0         4.0   5277.583333
1         5.0   3143.942568
2         6.0   3481.717322
3         7.0   8727.068211
4         8.0  30299.460967


The assumption macthes the analysis: the shows with the top 3 scores have the most amounts of votes.

## Conclusion <a id='hypotheses'></a>

The research done confirms that highly-rated shows released during the "Golden Age" of television also have the most votes. While shows with score 4 have more votes than ones with scores 5 and 6, the top three (scores 7-9) have the largest number. The data studied represents around 94% of the original set, so we can be confident in our findings.